In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook



In [3]:
cd /content/drive/My\ Drive/

/content/drive/My Drive


In [4]:
cd Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [5]:
cd data_workshop/

/content/drive/My Drive/Colab Notebooks/data_workshop


In [6]:
%ls

data/  day3.ipynb  day4.ipynb  HelloGithub.ipynb  LICENSE  README.md


In [7]:
ls data

men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values
  
  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [11]:
df['brand_cat'] = df['brand'].factorize()[0]
run_model(['brand_cat'], model = DecisionTreeRegressor(max_depth=7))

(-46.128313303883985, 1.713669567784974)

In [13]:
run_model(['brand_cat'], model = DecisionTreeRegressor(max_depth=19))

(-38.77514563076156, 2.4545340421435506)

In [14]:
run_model(['brand_cat'], model = RandomForestRegressor(max_depth=9, n_estimators=100, random_state=0))

(-41.8960891184654, 1.4999377040287982)

In [15]:
run_model(['brand_cat'], model = RandomForestRegressor(max_depth=19, n_estimators=100, random_state=0))

(-36.746144391693235, 2.2538470561354074)

In [16]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [20]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'], model = DecisionTreeRegressor(max_depth=7))

(-43.19525683629613, 1.8248434170356633)

In [19]:
df['brand_cat'] = df['brand'].factorize()[0]
run_model(['brand_cat'], model = DecisionTreeRegressor(max_depth=7))

(-46.128313303883985, 1.713669567784974)

In [25]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'], model = RandomForestRegressor(max_depth=15, n_estimators=100, random_state=0))

(-37.78584230550028, 2.337332035912339)

In [24]:
df['brand_cat'] = df['brand'].factorize()[0]
run_model(['brand_cat'], model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0))

(-45.176880678099224, 1.0791832177210356)

In [31]:
df.features.head

<bound method NDFrame.head of 0        [{"key":"Gender","value":["Men"]},{"key":"Shoe...
1        [{"key":"Gender","value":["Men"]},{"key":"Shoe...
2        [{"key":"Gender","value":["Men"]},{"key":"Colo...
3        [{"key":"Gender","value":["Men"]},{"key":"Colo...
4        [{"key":"Gender","value":["Men"]},{"key":"Colo...
                               ...                        
17592    [{"key":"Style","value":["Desert Boots"]},{"ke...
17593    [{"key":"Shoe Size","value":["9.5"]},{"key":"G...
17594    [{"key":"Gender","value":["Unisex"]},{"key":"F...
17595    [{"key":"Gender","value":["Unisex"]},{"key":"F...
17596    [{"key":"Gender","value":["Unisex"]},{"key":"F...
Name: features, Length: 17597, dtype: object>

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    output_dict[key] = value
  
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)


In [39]:
df.features_parsed.head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [40]:
keys = set()
df['features_parsed'].map(lambda x: keys.update(x.keys()))

0        None
1        None
2        None
3        None
4        None
         ... 
17592    None
17593    None
17594    None
17595    None
17596    None
Name: features_parsed, Length: 17597, dtype: object

In [41]:
len (keys)

469

In [42]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan )

In [43]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_interior pockets', 'feat_light transmission (vlt)',
       'feat_footwear type', 'feat_shirt size', 'feat_dimensions',
       'feat_article', 'feat_series name', 'feat_pattern', 'feat_band type',
       'feat_enhanced visibility (reflective)'],
      dtype='object', length=519)

In [44]:
df[df['feat_athlete'].isnull()].shape

(17589, 519)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [47]:
keys_stat

{'100% adidas authentic': 0.005682786838665682,
 'accessory type': 0.1477524578053077,
 'adidas': 0.005682786838665682,
 'adjustable': 0.2159458998692959,
 'age': 0.4546229470932545,
 'age end': 0.4603057339319202,
 'age gender group': 0.13070409728931068,
 'age group': 28.328692390748422,
 'age range': 0.5682786838665682,
 'age segment': 0.1477524578053077,
 'age start': 0.4603057339319202,
 'airport friendly': 0.005682786838665682,
 'alarm': 0.1818491788373018,
 'amazonbestsellersrank': 0.056827868386656814,
 'animal type': 0.005682786838665682,
 'antiscratch lens coating': 0.005682786838665682,
 'applicable': 0.022731147354662726,
 'arm': 0.6307893390918907,
 'article': 0.005682786838665682,
 'assembled in country of origin': 2.25038358811161,
 'assembled product dimensions (l x w x h)': 10.569983519918168,
 'assembled product weight': 1.568449167471728,
 'athlete': 0.04546229470932545,
 'atpv arc rating': 0.005682786838665682,
 'attachment': 0.03409672103199409,
 'audience': 0.2443

In [52]:
{k:v for k, v in keys_stat.items() if v > 33}

{'brand': 49.70165369097005,
 'color': 48.803773370460874,
 'gender': 51.16212990850714,
 'manufacturer part number': 37.02903904074558,
 'material': 35.585611183724495}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_manufacturer_part_number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]



In [55]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[df.brand == df.feat_brand].shape

(8704, 524)

In [65]:
model = RandomForestRegressor(max_depth=15, n_estimators=100 )
run_model(['brand_cat'])

(-44.54432445485899, 1.1253159560081818)

In [59]:
run_model(['brand_cat', 'feat_brand_cat'], model)

(-43.837796272605615, 1.3956301696970768)

In [60]:
run_model(['brand_cat', 'feat_brand_cat', 'feat_color_cat'], model)

(-44.946276763667605, 1.3388427770382914)

In [66]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_manufacturer_part_number_cat']
run_model(feats, model)

(-38.82332582665548, 2.8058621012719756)

In [67]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=15, n_estimators=100, random_state=0)
m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X, y)
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.5854 ± 0.0146,brand_cat
0.3698 ± 0.0121,feat_material_cat
0.3023 ± 0.0109,feat_color_cat
0.2085 ± 0.0041,feat_brand_cat
0.1492 ± 0.0062,feat_gender_cat
0.0894 ± 0.0026,feat_manufacturer_part_number_cat


In [71]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_material_cat']
run_model(feats, model)

(-37.9472061420141, 2.420908302329625)

In [72]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=15, n_estimators=100, random_state=0)
m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X, y)
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.6622 ± 0.0077,brand_cat
0.3786 ± 0.0181,feat_material_cat
0.3406 ± 0.0096,feat_color_cat
0.2470 ± 0.0022,feat_brand_cat
